<a href="https://colab.research.google.com/github/zzwony/Start_0920/blob/main/01_03_object_detection_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# git clone 으로 데이터 로드
# cd -> change directory
# pip -> 필요한거 설치
# yolov5는 torch로 진행된다.

# 목표: 안전모를 검출하기

# hhw라는 빈 폴더를 만들어주었다.

# -L은 리다이렉션이다.
# zip파일을 풀고 제거해주었다.
# 그러면 train, test가 생성된다.

# glob를 이용해서 list로 변환해주었다.
# 이미지들이 다 jpg로 되어있는데
# 이걸 glob를 이용해서 리스트의 형태로 바꿔주었다.

# 여기서 valid가 없어서 train_test_split를 사용하여
# test와 val을 나눠주었다.
# (train은 그냥 학습하고 test를 가지고 나눴다.)
# 그럼 train, valid, test가 되었다.

# 파일 구성을 나타내는 yaml이다.
# with open 해서 train, test,val.txt를 만들어주었다.
# 각각의 리스트에 대행하게끔 txt 파일을 만들어주었다.
# (hhw에 만들어진다.)


# 데이터 구성을 보기 위해 yaml을 사용함
# 들여다보고 필요한 조치를 취할것임.
# (decorate에 매직 명령어를 넣음)

# 안전모 데이터에 데이터 yaml을 실행시켜서 보니
# 우리는 train, test, val 이렇게 전에 만들어 두었다.
# 만들어둔걸로 수정을 함(writetemplate)

# %cat 으로 확인하면 변경사항이 잘 적용됌.

# 모델 구성에서
# num_classes에 담아주고 yaml 파일을 열어서 확인하기
# 들어가서 보니까 nc가 80이다.
# 우리는 위에서 nc가 3이였으니 수정이 필요하다.

# 어떻게 바꾸냐면 마찬가지로
# writetemplate을 사용하여 수정을 하면 된다.
# 위에 있는걸 그대로 카피해서 붙여넣고 nc 부분만
# {num_classes}를 적용하면 된다.
# 적용시키고 확인(%cat)하면 잘 적용 되었다.

# 그럼 이제 yolov5와 안전모 데이터의 추가적인 학습이 가능해진다.

# 학습하는 것이기 때문에 train 파일을 사용한다.
# 학습에서 가중치는 업데이트가 되기 때문에 '' 이렇게 빈칸으로 비워두었다.

# 결과가 올라가는 추세면 에폭을 더 해볼 필요가 있다.


# 추론은 detect.py를 사용한다.


# YOLOv5

주차공간 데이터셋

https://public.roboflow.com/object-detection/pklot


In [2]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 14918, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14918 (delta 2), reused 0 (delta 0), pack-reused 14908
Receiving objects: 100% (14918/14918), 13.91 MiB | 18.62 MiB/s, done.
Resolving deltas: 100% (10246/10246), done.
/content/yolov5
     |████████████████████████████████| 184 kB 33.4 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 1.6 MB 83.7 MB/s 


In [3]:
%mkdir /content/yolov5/park
%cd /content/yolov5/park

/content/yolov5/park


In [4]:
!pwd

/content/yolov5/park


In [ ]:
!curl -L "https://public.roboflow.com/ds/m26uZ1s6Hm?key=QacEBdvaBg" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [6]:
from glob import glob

train_img_list = glob('/content/yolov5/park/train/images/*jpg')
test_img_list = glob('/content/yolov5/park/test/images/*jpg')
valid_img_list = glob('/content/yolov5/park/valid/images/*jpg')

In [7]:
with open('/content/yolov5/park/train/.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')

with open('/content/yolov5/park/test/.txt', 'w') as f:
  f.write('\n'.join(test_img_list)+'\n')

with open('/content/yolov5/park/valid/.txt', 'w') as f:
  f.write('\n'.join(valid_img_list)+'\n')

In [8]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [9]:
import yaml
%cat /content/yolov5/park/data.yaml

train: ../train/images
val: ../valid/images

nc: 2
names: ['space-empty', 'space-occupied']

In [10]:
%%writetemplate /content/yolov5/park/data.yaml
train: ./park/train/images
test: ./park/test/images
val: ./park/valid/images

nc: 2
names: ['space-empty', 'space-occupied']

In [11]:
%cat /content/yolov5/park/data.yaml

train: ./park/train/images
test: ./park/test/images
val: ./park/valid/images

nc: 2
names: ['space-empty', 'space-occupied']


In [12]:
import yaml
with open('/content/yolov5/park/data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [13]:
num_classes

'2'

In [14]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [15]:
%cat /content/yolov5/models/custom_yolov5s.yaml


# Parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, Fa

In [16]:
!pwd

/content/yolov5/park


In [ ]:
%%time
%cd /content/yolov5
!python train.py --img 640 --batch 32 --epochs 50 --data ./park/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pothole_results --cache

/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=./park/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=pothole_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-55-g632bf48 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, 

In [18]:
%load_ext tensorboard
%tensorboard --logdir runs

<IPython.core.display.Javascript object>

In [ ]:
!ls /content/yolov5/runs/train/pothole_results

In [ ]:
Image(filename='/content/yolov5/runs/train/pothole_results/train_batch0.jpg', width=1000)

In [ ]:
# 검증
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./park/data.yaml --img 640 --iou 0.65 

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/pothole_results/weights/best.pt --img 640 --conf 0.4 --source ./pothole/test/images

In [ ]:
# 저장된 곳에서 랜덤으로 뽑아서 볼것이다.
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'))
display(Image(filename=image_name))